## ![kb_logo](http://www.newslock.co.kr/news/photo/202107/43984_39727_5125.jpg) 제 3회 Future Finance A.I. Challenge 

* 구현 환경

> Windows 10

> Python = 3.6

> Tensorflow = 2.4.0

> Opencv = 4.5.3

### 라이브러리 로드
***

In [1]:
import cv2 
import numpy as np
import dlib
import tensorflow as tf
from tensorflow.keras.models import load_model
import cvlib as cv
import os
import webbrowser
import face_recognition as fr
from scipy.spatial import distance

### GPU 활성화 ( GPU 이용 시)
***

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


1 Physical GPUs, 1 Logical GPUs


### 파라메터, 변수 선언
***

In [3]:
detector = dlib.get_frontal_face_detector() # 안면 인식 dlib 모델 
predictor = dlib.shape_predictor('Add-on/shape_predictor_68_face_landmarks.dat') # 안면 Landmark dlib 모델 

In [4]:
model = load_model('Add-on/EyeDetector.h5') # 눈동자 인식 모델
mask_model = load_model('Add-on/Mask_Face_Detector.h5') # 마스크 연령 인식 모델 
browser_path = 'C:/Program Files (x86)/Microsoft/Edge/Application/msedge.exe %s' # 브라우저 실행 경로
USER_POS_THRESHOLD = 25 #유저의 안면 움직임이라고 판단할 수 있는 임계값
EYE_MARGIN = 1 # 인공지능 모델에 넣기 전 눈의 여백
PREDICT_AGE_THRESHOLD = 100 # 나이를 확정하기 전 필요한 프레임의 길이
PATIENCE_THRESHOLD = 5 # 영상에서 User가 인식되지 않았을 때 대기하는 프레임
age_list = ['kid','adult','old'] # 나이 인식 카테고리

In [15]:
cam = cv2.VideoCapture(0) # 동영상 로드
img = cv2.imread('Dataset/idcard_bsh.jpg')# 주민등록증 이미지 로드 
caution = cv2.imread('Dataset/caution.png') # 주의 창 이미지
background = cv2.imread('Dataset/white_background.jpg') # 로그 백그라운드

img = cv2.resize(img,dsize=(350,450),interpolation=cv2.INTER_LINEAR)
caution = cv2.resize(caution,dsize=(0,0),fx=0.5,fy=0.5,interpolation=cv2.INTER_LINEAR)
enc = fr.face_encodings(img) # 이미지 인코딩 

### Main Activity
***

In [ ]:

# 창 생성 및 위치 세팅
cv2.namedWindow('Log')
cv2.namedWindow('frame')
cv2.namedWindow('USER')
cv2.moveWindow(winname='frame', x=910, y=0)
cv2.moveWindow(winname='USER', x=910, y=550)
cv2.moveWindow(winname='Log', x=550, y=0)

# 변수 초기화
frame_index = 0 # 영상 프레임 번호 
patience=0 # 대기 한도
last_enc = 0 # ID 인식률
caution_patience = 0  
caution_status = False
CAUTION_THRESHOLD = 5
ID_CHECK_MODE = False # 민증 대조 확인 기능 (On/Off)
predict_age_list = np.array((0,0,0)) 


#얼굴 인식 후 STM 기능 시작
while True: 
    status , frame = cam.read()
    if not status: # Video Not Loaded
        print('Cam Not Loaded')
        break
    faces , conf = cv.detect_face(frame) #프레임 속 얼굴 인식 
    if faces:
        L,T,R,B = faces[0] # 인식된 얼굴의 좌표값 반환 
        L,T,R,B = [0 if val<0 else val for val in (L,T,R,B)] # 음수 값 제거 
        break
    print('no face detected... waiting..') 

    
while cam.isOpened():
    frame_index += 1 
    status , frame = cam.read()
    if not status: # Video Not Loaded 
        break
    faces , conf = cv.detect_face(frame)
    if not faces :
        print('no face detected')
        continue
    background = cv2.imread('Dataset/white_background.jpg')
    cv2.imshow('Log',background)
        
    User_or_Peek = [] # User(사용자) 와 Peeker(비사용자) 구분 리스트 
    for face in faces:
        l,t,r,b = face
        l,t,r,b = [0 if val<0 else val for val in face]
        User_or_Peek.append(((face[3]-face[1]) * (face[2] - face[0]),l,t,r,b)) # 얼굴 인식 범위 및 좌표 추가 
        User_or_Peek.sort(reverse=True) # 얼굴 인식 범위를 기준으로 크기 정렬 
        
    #print(User_or_Peek) # [디버그] 좌표 출력 
    _,l,t,r,b = User_or_Peek[0]
    
    # User의 좌표가 갑작스럽게 변경되었을 때다른 사람을 User로 인식함을 방지하고 User 좌표를 트래킹함
    if (distance.euclidean((l,t,r,b),(L,T,R,B)) > USER_POS_THRESHOLD) and patience < PATIENCE_THRESHOLD: 
        patience+=1
        Another_User_Check_Count =  len(User_or_Peek)
        for j in User_or_Peek[1:Another_User_Check_Count-1]:
            _,l,t,r,b = j
            if (distance.euclidean((l,t,r,b),(L,T,R,B)) > USER_POS_THRESHOLD):
                L,T,R,B = L+int((l-L)/2),T+int((t-T)/2),R+int((r-R)/2),B+int((b-B)/2)
                patience = 0
                break
        if patience:
            print('no USER detected..patience : {}'.format(patience))
            continue
    elif (distance.euclidean((l,t,r,b),(L,T,R,B)) > USER_POS_THRESHOLD) and patience >= PATIENCE_THRESHOLD:
        print('finding new face')
        L,T,R,B = l,t,r,b
    else :
        L,T,R,B = L+(l-L)//3,T+(t-T)//2,R+(r-R)//2,B+(b-B)//2
        patience = 0
    
    #유저 식별
    USER = np.copy(frame[t:b,l:r]) 
    cv2.imshow('USER',USER)
    cv2.rectangle(frame, (l,t), (r,b) , (0,128,255), 3) # USER인식 범위 출력
    
    # 이용자의 나이를 유추하고 그에 맞는 웹 브라우저를 로드
    ## 예측 전
    if frame_index < PREDICT_AGE_THRESHOLD: 
        # 마스크가 착용된 사용자의 안면 이미지 전처리 
        USER = USER/255. 
        USER = cv2.resize(USER,(96,96),interpolation=cv2.INTER_AREA)
        USER = cv2.cvtColor(USER.astype('float32'),cv2.COLOR_BGR2RGB)
        USER = USER.reshape([1,96,96,3])

        predict_age = mask_model.predict(USER) # 학습 모델을 통해 예측
        if not predict_age_list.any() :
            predict_age_list = np.ravel(predict_age)
        else :
            predict_age_list = sum((predict_age_list,np.ravel(predict_age))) # 사용자 안면에 대한 어린이,성인,노인 확률을 리스트에 추가
        
    ## 예측 됨
    elif frame_index == PREDICT_AGE_THRESHOLD:
        age = age_list[predict_age_list.argmax()] # 가장 확률이 높은 연령을 반환 
        
        # 어린이 , 성인 , 노인에 해당하는 웹 브라우저 (STM기의 화면) 로드 
        if age == 'kid':
            webbrowser.get(browser_path).open('file:///'+os.getcwd()+'/Web/child.html')
        elif age == 'old':
            webbrowser.get(browser_path).open('file:///'+os.getcwd()+'/Web/old.html')
        else:
            webbrowser.get(browser_path).open('file:///'+os.getcwd()+'/Web/adult.html')
            
            
    ## 예측 후 (맞춤형 웹 외 추가 활용 가능)
    else:
        pass
        #cv2.putText(frame,str(age),(100,60),cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,1,(0,0,255))
        

    # 민증 인식 , 상대적인 지표이며 80% 보다 높을 때 같은 사람으로 유추 가능 
    if ID_CHECK_MODE:
        cv2.imshow('ID_card',img) 
        cv2.moveWindow(winname='ID_card', x=1420, y=0)
        ID_MARGIN = min(l,t,r,b)
        idd = np.copy(frame[t-ID_MARGIN:b+ID_MARGIN,l-ID_MARGIN:r+ID_MARGIN])
        cv2.putText(background,'ID CHECK MODE ',(10,350),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
        frame_enc = fr.face_encodings(np.copy(frame[t-ID_MARGIN:b+ID_MARGIN,l-ID_MARGIN:r+ID_MARGIN]))
        if not frame_enc:
            if not last_enc:
                cv2.putText(background,'Detecting...',(10,390),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2)
            else:
                cv2.putText(background,'Similarity :{:.2f}%'.format(last_enc),(10,390),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2)
        else:
            dist = fr.face_distance(enc,frame_enc[0])
            dist = (1.2 - dist[0]) * 100
            cv2.putText(background,'Similarity :{:.2f}%'.format(dist),(10,390),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2)
            last_enc = dist

    User_or_Peek.pop(0)

    
    index=-1 # Peeker 의 수에 따른 index
    if not User_or_Peek:
        cv2.putText(background,'No Peeker',(10,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
        if (caution_patience == -CAUTION_THRESHOLD) and caution_status:
            cv2.namedWindow('caution')
            cv2.destroyWindow('caution')
            caution_status = False
            caution_patience = 0
        elif (caution_patience != -CAUTION_THRESHOLD):
            caution_patience -=1
            
    #비사용자 식별
    while User_or_Peek:
        index+=1
        if index >= 10: #10명 이상인 경우는 에러에 가까우므로 예외처리
            print('Peeker Detection Error')
            break
        _,l,t,r,b = User_or_Peek[0]
        PEEK = np.copy(frame[t:b,l:r]) 
        cv2.imshow('Peek_'+str(index),PEEK)
        cv2.moveWindow(winname='Peek_'+str(index), x=1150, y=550+(150*index))
        cv2.rectangle(frame, (l,t), (r,b) , (20,20,230), 2) # Peeker 인식 범위 출력

        landmark = predictor(frame,dlib.rectangle(l,t,r,b)) #얼굴의 눈 좌표를 특정하기 위한 Landmark 
        landmark_list = []
        for i,point in enumerate(landmark.parts()[36:48]): # 양쪽 눈 각각 6개씩 12개의 Face Point를 반환
            landmark_list.append([point.x,point.y])
        Eye_1_start = (landmark_list[0][0] , landmark_list[1][1])
        Eye_1_end = (landmark_list[3][0] , landmark_list[4][1])
        Eye_1_Height = np.mean((landmark_list[4][1] - landmark_list[2][1],landmark_list[5][1] - landmark_list[1][1]))
        Eye_2_start = (landmark_list[6][0] , landmark_list[7][1])
        Eye_2_end = (landmark_list[9][0] , landmark_list[10][1])
        Eye_2_Height = np.mean((landmark_list[11][1] - landmark_list[7][1],landmark_list[10][1] - landmark_list[8][1]))
        
        #양쪽 눈 이미지 추출
        eye1 = np.copy(frame[Eye_1_start[1]-EYE_MARGIN:Eye_1_end[1]+EYE_MARGIN,Eye_1_start[0]-EYE_MARGIN:Eye_1_end[0]+EYE_MARGIN])
        eye2 = np.copy(frame[Eye_2_start[1]-EYE_MARGIN:Eye_2_end[1]+EYE_MARGIN,Eye_2_start[0]-EYE_MARGIN:Eye_2_end[0]+EYE_MARGIN])
        
        if (not eye1.size) or (not eye2.size):
            print('Eye Size Error')
            index -= 1
            continue
        
        #인공지능 모델 예측을 위한 전처리
        eye1 = eye1/255.
        eye2 = eye2/255.
        eye1 = cv2.resize(eye1,(65,35),interpolation=cv2.INTER_AREA)
        eye1 = cv2.cvtColor(eye1.astype('float32'),cv2.COLOR_BGR2RGB)
        eye2 = cv2.resize(eye2,(65,35),interpolation=cv2.INTER_AREA)
        eye2 = cv2.cvtColor(eye2.astype('float32'),cv2.COLOR_BGR2RGB)
        cv2.imshow('left_eye_'+str(index),eye1)
        cv2.imshow('right_eye_'+str(index),eye2)
        cv2.moveWindow(winname='left_eye_'+str(index), x=1280, y=550+(150*index))
        cv2.moveWindow(winname='right_eye_'+str(index), x=1280, y=620+(150*index))
        eye1 = eye1.reshape([1,35,65,3])
        eye2 = eye2.reshape([1,35,65,3])
        

        # 쳐다보고 있는지 예측
        predict = model.predict(eye1)
        predict2 = model.predict(eye2)
            
        cv2.putText(background,'{} Peeker'.format(index+1),(10,70+(120*index)),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2)
        cv2.putText(background,'left eye : {:.1f}%'.format(predict[0][0]*100),(10,110+(120*index)),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2)
        cv2.putText(background,'right eye : {:.1f}%'.format(predict2[0][0]*100),(10,150+(120*index)),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2)
        #cv2.putText(frame,str(caution_patience),(10,200),cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,1,(0,0,255),)
        
        if ((predict > 0.7) or (predict2 > 0.7)) :
            cv2.putText(background,'WATCHING',(10,30),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            if caution_patience > 0:
                cv2.imshow('caution',caution)
                cv2.moveWindow(winname='caution', x=1650, y=0)
                caution_status = True
            if caution_patience < CAUTION_THRESHOLD:
                caution_patience += 1
        else:
            if (caution_patience == -CAUTION_THRESHOLD) and caution_status:
                cv2.destroyWindow('caution')
                caution_status = False
                caution_patience = 0
            elif (caution_patience != -CAUTION_THRESHOLD):
                caution_patience -=1
                
        User_or_Peek.pop(0)
        
    
    cv2.imshow('frame',frame)
    cv2.imshow('Log',background)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cam.release()
        cv2.destroyAllWindows()
    if cv2.waitKey(1) == ord('w'): 
        if not ID_CHECK_MODE:
            ID_CHECK_MODE = True
        else:
            ID_CHECK_MODE = False
cv2.destroyAllWindows()

no USER detected..patience : 1
no USER detected..patience : 2
no USER detected..patience : 3
no USER detected..patience : 4
no USER detected..patience : 5
finding new face
no USER detected..patience : 1
no USER detected..patience : 2
no USER detected..patience : 1
no USER detected..patience : 1
no USER detected..patience : 2
no USER detected..patience : 3
no USER detected..patience : 4
no USER detected..patience : 1
no USER detected..patience : 2
no USER detected..patience : 3
no USER detected..patience : 4
no USER detected..patience : 1
no USER detected..patience : 2
no USER detected..patience : 3
no USER detected..patience : 1
no USER detected..patience : 2
no USER detected..patience : 3
no USER detected..patience : 4
no USER detected..patience : 5
finding new face
no USER detected..patience : 1
no USER detected..patience : 2
no USER detected..patience : 1
no USER detected..patience : 2
no USER detected..patience : 3
no USER detected..patience : 4
no USER detected..patience : 5
findi